# Working with regions.

Often we may need to break the level of analysis down to a more granular spatial level of resolution. 

For example, extracting from national data layers may be too computationally (and thus time) intensive.

It is possible to break down all analysis into regions by using a regional loop at each step of the analysis. This may involve:

- Extracting a national boundary.
- Cutting up raster data using the national boundary to produce a national raster layer.
- Cutting up the national raster layer to each region. 
- Extracting data for each region. 

## Raster clipping example

In the example below, we will work with a single country (e.g., Rwanda). 

This will involve loading in our region shapes which we previously processed, and then iterating over each region one at a time. 

The underlying global raster will be clipped for each region, and then exported. Each region can then be accessed separately later.

The code has been annotated to support your understanding. While similar to other code, the key point is that the regions need to be loaded in within the country loop. And to do that successfully (as we want to specify different regional levels), we need to use:

- a `gid_region` variable which states the gid regional level we want to use (e.g., 0, 1 or 2).
- a `gid_level` variable which produces a string for the gid level (e.g., 'GID_1' or 'GID_2').
- a `gid_id` variable which holds the underlying unique regional code reference (e.g., RWA.1_1).


In [ ]:
# Example
import os
import json
import rasterio
from rasterio.mask import mask
import pandas
import geopandas as gpd

path = os.path.join('..', 'data', 'countries.csv')
countries = pandas.read_csv(path, encoding='latin-1')

for idx, country in countries.iterrows():

    if not country['iso3'] == 'RWA': # let's work on a single country at a time
        continue   
    
    #define our country-specific parameters, including gid information
    iso3 = country['iso3']
    gid_region = country['gid_region']
    gid_level = 'GID_{}'.format(gid_region)
    
    #set the filename depending our preferred regional level
    filename = "gadm36_{}.shp".format(gid_region)
    folder = os.path.join('..', 'data','processed', iso3, 'regions')
    
    #then load in our regions as a geodataframe
    path_regions = os.path.join(folder, filename)
    regions = gpd.read_file(path_regions, crs='epsg:4326')#[:2]
    
    for idx, region in regions.iterrows():

        #get our gid id for this region 
        #(which depends on the country-specific gid level)
        gid_id = region[gid_level]
        
        print('----Working on {}'.format(gid_id))
        
        #let's load in our hazard layer
        filename = 'inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000.tif'
        path_hazard = os.path.join('..', 'data','raw','flood_hazard', filename)
        hazard = rasterio.open(path_hazard, 'r+')
        hazard.nodata = 255                       #set the no data value
        hazard.crs.from_epsg(4326)                #set the crs

        #create a new gpd dataframe from our single region geometry
        geo = gpd.GeoDataFrame(gpd.GeoSeries(region.geometry))
        #this line sets geometry for resulting geodataframe
        geo = geo.rename(columns={0:'geometry'}).set_geometry('geometry')
        #convert to json
        coords = [json.loads(geo.to_json())['features'][0]['geometry']] 
        
        #carry out the clip using our mask
        out_img, out_transform = mask(hazard, coords, crop=True)
        out_img, out_transform

        #update our metadata
        out_meta = hazard.meta.copy()
        out_meta.update({"driver": "GTiff",
                        "height": out_img.shape[1],
                        "width": out_img.shape[2],
                        "transform": out_transform,
                        "crs": 'epsg:4326'})

        #now we write out at the regional level
        filename_out = '{}.tif'.format(gid_id) #each regional file is named using the gid id
        folder_out = os.path.join('..', 'data', 'processed', 'RWA', 'hazards', 'inunriver', gid_id)
        if not os.path.exists(folder_out):
            os.makedirs(folder_out)
        path_out = os.path.join(folder_out, filename_out)

        with rasterio.open(path_out, "w", **out_meta) as dest:
            dest.write(out_img)
    print('--')
    print('Processing complete for {}'.format(iso3))


This method can easily be adapted for other coding tasks, enabling data input, processing and export to be undertaken at the regional level.